In [2]:

def process_raster(raster_path, polygon_gdf):
    with rasterio.open(raster_path) as src:
        data = src.read(1)
        if data.dtype not in ['int16', 'int32', 'uint8', 'uint16', 'float32']:
            data = data.astype('float32')   
        crs = src.crs
        transform = src.transform
        data = np.where(data > 0, 1, 0).astype('uint8')
        mask = data == 1
        feature_generator = shapes(data, mask=mask, transform=transform)
        polygons = [shape(feature[0]) for feature in feature_generator]
        gdf = GeoDataFrame({'geometry': polygons}, crs=crs)    
        polygon_gdf = polygon_gdf.to_crs(crs)
        intersection_gdf = gpd.overlay(gdf, polygon_gdf, how='intersection')
        return len(intersection_gdf)
def compare_rasters(benchmark_path, candidate_paths, polygon_gdf):
    benchmark_result = process_raster(benchmark_path, polygon_gdf)
    candidate_results = [process_raster(path, polygon_gdf) for path in candidate_paths]
    return benchmark_result, candidate_results

benchmark_result, candidate_results = compare_rasters(benchmark_path, candidate_paths, polygon_gdf)
